In [1]:
!pip install transformers
!pip install torch
!pip install -U datasets

In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Convert the dataset text to a csv:

In [3]:
text_filepath = "../home/schen9/dataset.txt"
csv_filepath = "dataset.csv"

In [5]:
with open(text_filepath, "r", encoding="utf-8") as text_file:
    rows = text_file.read().strip().split("\n\n")

In [7]:
import csv

with open(csv_filepath, "w", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Text"])
    for i, row in enumerate(tqdm(rows)):
        writer.writerow([row.strip()])

print(f"Finished writing to {csv_filepath}") 

  0%|          | 0/449919 [00:00<?, ?it/s]

Finished writing to dataset.csv


In [8]:
from datasets import Dataset, load_dataset

dataset = load_dataset("csv",data_files=csv_filepath) 
dataset = dataset.filter(lambda x: x["Text"] is not None) # filter out NoneTypes
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/449919 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Text'],
        num_rows: 446203
    })
})

In [11]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

transformer_name = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(transformer_name, use_fast=True)

In [14]:
def tokenize(batch):
    return tokenizer(batch["Text"], truncation=True)

batch_size = 20
tokens = dataset.map(tokenize, batched=True, batch_size=batch_size)
tokens

Map:   0%|          | 0/446203 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Text', 'input_ids', 'attention_mask'],
        num_rows: 446203
    })
})

In [16]:
tokens["train"].to_pandas()

,Text,input_ids,attention_mask
0,The White Monkey is a 1925 American silent dra...,"[0, 133, 735, 34546, 16, 10, 36248, 470, 8454,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Preservation\nAn incomplete print of The White...,"[0, 28917, 26481, 50118, 4688, 20044, 5780, 9,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,External links,"[0, 47380, 5678, 2]","[1, 1, 1, 1]"
3,Films based on works by John Galsworthy\nFilms...,"[0, 36361, 4339, 716, 15, 1364, 30, 610, 272, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,the montane grasslands and shrublands biome\n ...,"[0, 627, 27121, 1728, 6964, 8391, 8, 15383, 17...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...
446198,References,"[0, 49379, 2]","[1, 1, 1]"
446199,External links,"[0, 47380, 5678, 2]","[1, 1, 1, 1]"
446200,"1929 births\n2015 deaths\nPeople from Naseby, ...","[0, 1646, 2890, 26906, 50118, 14420, 3257, 501...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
446201,Scottish female golfers\nGolfers from Edinburg...,"[0, 22041, 1173, 2182, 3524, 268, 50118, 534, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [17]:
l = ["yo","1","2"]
t = "".join(l)
t
print(type(l))

<class 'list'>
